# Imports

In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
pd.options.display.max_columns = None

# Lecture des données

In [35]:
fifa15 =pd.read_csv('../data/FIFA/players_15.csv',dtype={'gk':str})
fifa16 =pd.read_csv('../data/FIFA/players_16.csv',dtype={'gk':str})
fifa17 =pd.read_csv('../data/FIFA/players_17.csv')
fifa18 =pd.read_csv('../data/FIFA/players_18.csv', dtype= {'lw': str, 'lf': str, 'cf': str, 'rf': str, 'rw': str})
fifa19 =pd.read_csv('../data/FIFA/players_19.csv')
fifa20 =pd.read_csv('../data/FIFA/players_20.csv')
fifa21 =pd.read_csv('../data/FIFA/players_21.csv')
fifa22 =pd.read_csv('../data/FIFA/players_22.csv', dtype = {'nation_position':str, 'nation_logo_url':str})

In [36]:
clubs = pd.read_csv('../data/games/clubs.csv', index_col = 0)
leagues = pd.read_csv('../data/games/leagues.csv', index_col = 0)
games = pd.read_csv('../data/games/games.csv', index_col = 0,
                   dtype = {'game_id' : np.int32, 'season' : np.int16, 'home_club_id' : np.int32, 'home_club_id' : np.int32,
                           'home_club_goals' : np.int8, 'away_club_goals' : np.int8})

players = pd.read_csv('../data/games/players.csv', index_col = 0,
                     dtype = {'player_id' : np.int32, 'player_club_id' : np.int16})

appearance = pd.read_csv('../data/games/appearance.csv', index_col = 0)
corr = pd.read_csv('../data/table_correspondance/table_correspondance_finale.csv', index_col = 0)
player_IDs = corr.player_id.unique()


In [38]:
appearance_select =\
pd.read_csv('../data/training_data/appearance_select6.csv', index_col = 0, \
            dtype = {'game_id' : np.int32, 'player_id' : np.int32,'player_club_id' : np.int16, 'minutes_played' : np.int8,\
                    'yellow_cards' : np.int8,'red_cards' : np.int8,'goals' : np.int8,'assists' : np.int8,
                    'max_time' : np.int16,'time_ratio' : np.float16, 'season' : np.int8})

appearance_known_players =\
pd.read_csv('../data/training_data/appearance_known_players6.csv', index_col = 0,\
            dtype = {'game_id' : np.int32, 'player_id' : np.int32,'player_club_id' : np.int16, 'minutes_played' : np.int8,\
                    'yellow_cards' : np.int8,'red_cards' : np.int8,'goals' : np.int8,'assists' : np.int8,
                    'max_time' : np.int16,'time_ratio' : np.float16, 'season' : np.int8})

train1 = pd.read_csv('train1_6.csv', index_col = 0)

#### colonnes des statistiques fifa à sélectionner

In [39]:
stats_col = ['overall' ,'potential','pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing','attacking_heading_accuracy','skill_curve','skill_fk_accuracy', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power','power_jumping', 'power_stamina','mentality_aggression',
       'mentality_positioning', 'mentality_penalties','mentality_composure']


GK_stats_col = ['overall' ,'potential','attacking_crossing','attacking_heading_accuracy','skill_curve','skill_fk_accuracy', 
                'movement_agility','movement_reactions', 'movement_balance', 'power_shot_power','power_jumping', 'power_stamina',
                'mentality_aggression','mentality_positioning', 'mentality_penalties','goalkeeping_diving', 
                'goalkeeping_speed']

#### Identification des quelques matchs sans GK

In [40]:

bad_game_ids = train1.loc[train1.GK < 0.8,'game_id'].unique()

# Création d'un profil moyen de joueurs d'attaque/défense/milieu/GK

In [41]:
# Arrangeons un petit peu le df players
players.loc[players.player_id == 290801,'position'] = 'Defender'
players.loc[players.player_id == 290801,'sub_position'] = 'Centre-Back'
t  = players.loc[players.position == '0','sub_position']
players.loc[t.index,'position'] = t
players.drop(columns =
             ['name','pretty_name','country_of_birth','country_of_citizenship','foot','height_in_cm','market_value_in_gbp' ,'highest_market_value_in_gbp' ,'url'],
            inplace = True)
players.position = players.position.apply(lambda x: x.lower() )


In [42]:
# liste des joueurs de la table de correspondance pouvant être trouvés dans fifa22:
players22_ID = corr.loc[corr.which_fifa==22,'player_id'].values

# faisons un df ne contenant que ces joueurs
players22 = players.loc[players.player_id.isin(players22_ID),:].reset_index(drop=True).copy()

In [43]:
players22.position.unique()

array(['defender', 'midfield', 'goalkeeper', 'attack'], dtype=object)

In [44]:
GK22 = players22.loc[players22.position == 'goalkeeper',:].reset_index(drop=True).copy()
MID22 = players22.loc[players22.position == 'midfield',:].reset_index(drop=True).copy()
DEF22 = players22.loc[players22.position == 'defender',:].reset_index(drop=True).copy()
ATK22 = players22.loc[players22.position == 'attack',:].reset_index(drop=True).copy()

## GK Moyen

In [45]:
def seek_statGK22(x):
    sofID = corr.loc[(corr.player_id == x) & (corr.which_fifa == 22),'sofifa_id'].values[0]
    return fifa22.loc[fifa22.sofifa_id==sofID,GK_stats_col]

tt=GK22.player_id.progress_apply(seek_statGK22)
all_GK = pd.concat(tt.values.tolist())
####====>
avg_GK = all_GK.mean(axis=0)

100%|██████████| 880/880 [00:01<00:00, 621.67it/s]


## ATK, DEF, & MID

In [46]:
def seek_stat22(x):
    sofID = corr.loc[(corr.player_id == x) & (corr.which_fifa == 22),'sofifa_id'].values[0]
    return fifa22.loc[fifa22.sofifa_id==sofID,stats_col]

tt=ATK22.player_id.progress_apply(seek_stat22)
all_ATK = pd.concat(tt.values.tolist())
avg_ATK = all_ATK.mean(axis=0)


tt=DEF22.player_id.progress_apply(seek_stat22)
all_DEF = pd.concat(tt.values.tolist())
avg_DEF = all_DEF.mean(axis=0)

tt=MID22.player_id.progress_apply(seek_stat22)
all_MID = pd.concat(tt.values.tolist())
avg_MID = all_MID.mean(axis=0)

100%|██████████| 2387/2387 [00:03<00:00, 617.65it/s]


# stats des GK

In [47]:
#On commence par retirer les matchs ou il n'y a pas de GK parce que c'est bizarre

appearance_select = \
appearance_select.drop(appearance_select.loc[appearance_select.game_id.isin(bad_game_ids),:].index).reset_index(drop = True)

appearance_known_players = \
appearance_known_players.drop(appearance_known_players.loc[appearance_known_players.game_id.isin(bad_game_ids),:].index).reset_index(drop = True)

train1 = train1.drop(train1.loc[train1.game_id.isin(bad_game_ids),:].index).reset_index(drop = True)


In [48]:
# On créé un dataframe à partir de appearance_known_players avec uniquement les GKs

known_GK = appearance_known_players.loc[appearance_known_players.position == 'goalkeeper',
                                        ['game_id','player_club_id', 'time_ratio','which_fifa','player_id']].reset_index(drop = True).copy()
#On garde en tête qu'on a pu perdre des lignes voire des équipes dans des matches

In [49]:
# Test d'une ligne d'un apply:
#On extrait les stats (GK_stats_col) d'un player_id dans un dataframe fifa désigné par which_fifa 

x = known_GK.iloc[3,:]
locs = locals()
sofifa = corr.loc[(corr.player_id == x['player_id']) & (corr.which_fifa == x['which_fifa']), 'sofifa_id'].values[0]
FIFA = locs['fifa'+str(int(x['which_fifa']))]
FIFA.loc[FIFA.sofifa_id==sofifa,GK_stats_col]


,overall,potential,attacking_crossing,attacking_heading_accuracy,skill_curve,skill_fk_accuracy,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,mentality_aggression,mentality_positioning,mentality_penalties,goalkeeping_diving,goalkeeping_speed
263,80,84,25,25,25,25,42,79,37,28,76,45,30,25,35,81,49.0


In [50]:
#On extrait les stats (GK_stats_col) d'un player_id dans un dataframe fifa désigné par which_fifa 

locs = locals()
def GK_stats(x):
    sofifa = corr.loc[(corr.player_id == x['player_id']) & (corr.which_fifa == x['which_fifa']), 'sofifa_id'].values[0]
    FIFA = locs['fifa'+str(int(x['which_fifa']))]
    
    return FIFA.loc[FIFA.sofifa_id==sofifa,GK_stats_col]
    
    
    
GK_stats = known_GK.progress_apply(lambda x: GK_stats(x)
                          ,axis = 1
                         )
GK_stats = pd.concat(GK_stats.values.tolist()).reset_index(drop = True)

100%|██████████| 59459/59459 [01:46<00:00, 556.62it/s]


In [51]:
#on renomme les colonnes de GK_stats pour ajouter GK au début!
GK_stats.rename(columns = {x:'GK_'+x for x in GK_stats.columns}, inplace = True)
GK_stats.head(1)

,GK_overall,GK_potential,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed
0,80,84,25,25,25,25,42,79,37,28,76,45,30,25,35,81,49.0


In [52]:
# On multiplie les stats par le time_ratio (moyenne pondérée):
GK_stats = GK_stats.multiply(known_GK.time_ratio,axis = 0)

In [53]:
# On ajoute GK_stats à known_GK
if known_GK.shape[1]==5:
    known_GK = pd.concat([known_GK , GK_stats],axis = 1)

In [54]:
# On fait la somme des stats pondérées des gardiens de but par équipes et par match
known_GK = known_GK.groupby(['game_id','player_club_id'])[np.append(['time_ratio'],GK_stats.columns)].sum().reset_index()

In [55]:
# Puis on divise par la somme des time_ratios (toujours la moyenne pondérée):

known_GK.loc[:,GK_stats.columns] = known_GK.loc[:,GK_stats.columns].divide(known_GK.time_ratio, axis = 0)


In [56]:
# à ce stade on a des lignes pour les GK utilisables pour l'entrainement du modèle.
# Mais on sait que quand on n'a selectionné que les GK que l'on connait, on a perdu quelques lignes en route. 
# En l'occurence, comme il n'y a en général qu'un GK par match et par équipe, on a dû perdre des équipes. On vérifie:

team_count = known_GK.groupby('game_id')['player_club_id'].unique().reset_index()
print(team_count.loc[team_count.player_club_id.apply(len)<2,'game_id'].shape[0])
#il en manque 1910

1910


In [57]:
team_count.player_club_id = team_count.player_club_id.apply(lambda x: np.squeeze(x))
team_count.loc[team_count.player_club_id.apply(type)==int,:].head(1)


,game_id,player_club_id
1,2458528,660


In [58]:
# On créée le fichier d'entrainement avec pour l'instant seulement les GK.
# On fera la même chose pour les ATK, DEF et MF, puis on mergera tout ça
train_GK = train1.loc[:,['game_id','player_club_id','GK']].copy()

In [59]:
train_GK = pd.merge(train_GK , known_GK.drop(columns = ['time_ratio']), \
         on = ['player_club_id','game_id'], how = 'left')


In [60]:
train_GK.loc[train_GK.GK_overall.isnull(),:]

,game_id,player_club_id,GK,GK_overall,GK_potential,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed
2,2458528,338,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
628,2460301,281,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900,2460757,24,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
918,2460766,24,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
929,2460771,24,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60779,3673543,6676,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60782,3673545,653,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60803,3683209,371,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60840,3700970,-4587,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
train_GK.loc[train_GK.GK_overall.isnull(),GK_stats.columns] = avg_GK.values
train_GK.loc[train_GK.GK_overall == avg_GK.values[0],:].head()

,game_id,player_club_id,GK,GK_overall,GK_potential,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed
2,2458528,338,1.0,68.097727,72.825,13.936364,14.119318,14.730682,13.832955,41.163636,62.260227,40.938636,48.728409,57.7,30.297727,26.217045,10.523864,19.643182,68.715909,38.606818
628,2460301,281,1.0,68.097727,72.825,13.936364,14.119318,14.730682,13.832955,41.163636,62.260227,40.938636,48.728409,57.7,30.297727,26.217045,10.523864,19.643182,68.715909,38.606818
900,2460757,24,1.0,68.097727,72.825,13.936364,14.119318,14.730682,13.832955,41.163636,62.260227,40.938636,48.728409,57.7,30.297727,26.217045,10.523864,19.643182,68.715909,38.606818
918,2460766,24,1.0,68.097727,72.825,13.936364,14.119318,14.730682,13.832955,41.163636,62.260227,40.938636,48.728409,57.7,30.297727,26.217045,10.523864,19.643182,68.715909,38.606818
929,2460771,24,1.0,68.097727,72.825,13.936364,14.119318,14.730682,13.832955,41.163636,62.260227,40.938636,48.728409,57.7,30.297727,26.217045,10.523864,19.643182,68.715909,38.606818


In [62]:
train_GK.to_csv('../data/training_data/train_GK6.csv')

# stats des Attaquants

In [87]:
known_ATK = appearance_known_players.loc[appearance_known_players.position == 'attack',
                                        ['game_id','player_club_id', 'time_ratio','which_fifa','player_id']].reset_index(drop = True).copy()


In [89]:
locs = locals()
def player_stats(x):
    sofifa = corr.loc[(corr.player_id == x['player_id']) & (corr.which_fifa == x['which_fifa']), 'sofifa_id'].values[0]
    FIFA = locs['fifa'+str(int(x['which_fifa']))]
    
    return FIFA.loc[FIFA.sofifa_id==sofifa,stats_col]
    
    
    
ATK_stats = known_ATK.progress_apply(lambda x: player_stats(x)
                          ,axis = 1
                         )
ATK_stats = pd.concat(ATK_stats.values.tolist()).reset_index(drop = True)

100%|██████████| 224873/224873 [06:45<00:00, 554.67it/s]


In [90]:
ATK_stats.rename(columns = {x:'ATK_'+x for x in ATK_stats.columns}, inplace = True)
ATK_stats.head(1)

,ATK_overall,ATK_potential,ATK_pace,ATK_shooting,ATK_passing,ATK_dribbling,ATK_defending,ATK_physic,ATK_attacking_crossing,ATK_attacking_heading_accuracy,ATK_skill_curve,ATK_skill_fk_accuracy,ATK_movement_agility,ATK_movement_reactions,ATK_movement_balance,ATK_power_shot_power,ATK_power_jumping,ATK_power_stamina,ATK_mentality_aggression,ATK_mentality_positioning,ATK_mentality_penalties,ATK_mentality_composure
0,56,63,52.0,55.0,40.0,55.0,30.0,64.0,27,58,33,34,55,54,45,60,68,65,51,48,37,NaN


In [122]:
# avant de traiter les statistiques des attaquants pour faire une moyenne pondérée (multiply by time_ratio), 
# on garde les statistiques brutes afin de pouvoir aller chercher le meilleur attaquant de l'équipe
best_ATK = ATK_stats.rename(columns = {x:'best_'+x for x in ATK_stats.columns}).copy()

# On ajoute ATK_stats_raw à known_ATK
if best_ATK.shape[1] == 22:
    best_ATK = pd.concat([known_ATK , best_ATK],axis = 1)
best_ATK.head(1)

,best_ATK_overall,best_ATK_potential,best_ATK_pace,best_ATK_shooting,best_ATK_passing,best_ATK_dribbling,best_ATK_defending,best_ATK_physic,best_ATK_attacking_crossing,best_ATK_attacking_heading_accuracy,best_ATK_skill_curve,best_ATK_skill_fk_accuracy,best_ATK_movement_agility,best_ATK_movement_reactions,best_ATK_movement_balance,best_ATK_power_shot_power,best_ATK_power_jumping,best_ATK_power_stamina,best_ATK_mentality_aggression,best_ATK_mentality_positioning,best_ATK_mentality_penalties,best_ATK_mentality_composure
0,56,63,52.0,55.0,40.0,55.0,30.0,64.0,27,58,33,34,55,54,45,60,68,65,51,48,37,NaN


In [127]:
#créé un masque qui renvoie le(s) joueur(s) qui a la note overall la plus élevée
id_best = best_ATK.groupby(['game_id','player_club_id'])['best_ATK_overall'].transform(max) == best_ATK.best_ATK_overall
best_ATK = best_ATK[id_best]

In [128]:
# Vérifions si il y a des cas ou il y a plusieurs best_ATK par match et par équipe:
display(best_ATK.groupby(['game_id','player_club_id'])['time_ratio'].count().value_counts())
#un petit paquet quand même....
tt = best_ATK.groupby(['game_id','player_club_id'])['time_ratio'].count().reset_index()
multiple_best = tt.loc[tt.time_ratio !=1,:].drop(columns = ['time_ratio'])
best_ATK.loc[(best_ATK.game_id.isin(multiple_best.game_id)) & (best_ATK.player_club_id.isin(multiple_best.player_club_id)) , : ].head()
#on voit qu'il s'agit de joueurs qui apparaissent plusieurs fois par match dans le dataset appearance car ils sortent et re-rentrent.

1    53075
2     6879
3      637
4       50
Name: time_ratio, dtype: int64

In [130]:
#maintenant on fait un groupby.first() pour prendre la première ligne de chaque groupe
best_ATK = best_ATK.groupby(['game_id','player_club_id']).first().reset_index()
# et on le merge avec train, on drop time_ratio 
train_best_ATK = train1.loc[:,['game_id','player_club_id','attack']].copy()
train_best_ATK = pd.merge(train_best_ATK , best_ATK.drop(columns = ['time_ratio', 'which_fifa', 'player_id']), \
         on = ['player_club_id','game_id'], how = 'left')


In [152]:
# Reste deux cas de figures: soit un match n'a pas d'attaquant (c'est possible!), soit on ne connait pas les attaquants!
#S'il n'y a pas d'attaquants, alors attack=0 et on met les stats à 0
train_best_ATK.loc[train_best_ATK.attack == 0,train_best_ATK.columns[3:]]=0
display(train_best_ATK.loc[train_best_ATK.attack == 0,:].head(1))

#Pour les matchs où on ne connait aucun attaquant:
index = train_best_ATK.loc[train_best_ATK.best_ATK_overall.isnull(),:].index
train_best_ATK.loc[index,train_best_ATK.columns[3:]] = avg_ATK.values
train_best_ATK.loc[index,:].head(2)

,game_id,player_club_id,attack,best_ATK_overall,best_ATK_potential,best_ATK_pace,best_ATK_shooting,best_ATK_passing,best_ATK_dribbling,best_ATK_defending,best_ATK_physic,best_ATK_attacking_crossing,best_ATK_attacking_heading_accuracy,best_ATK_skill_curve,best_ATK_skill_fk_accuracy,best_ATK_movement_agility,best_ATK_movement_reactions,best_ATK_movement_balance,best_ATK_power_shot_power,best_ATK_power_jumping,best_ATK_power_stamina,best_ATK_mentality_aggression,best_ATK_mentality_positioning,best_ATK_mentality_penalties,best_ATK_mentality_composure
5160,2486624,281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [157]:
#train_best_ATK.drop(columns = ['game_id'	,'player_club_id'	,'attack'	,'best_ATK_overall']).info()
# Plein de valeurs manquantes dans 'mentality composure.... on supprime cette colonne
train_best_ATK.drop(columns= ['best_ATK_mentality_composure'], inplace = True)

In [161]:
train_best_ATK.to_csv('../experimentation/data/training_data/train_best_ATK.csv')


In [163]:
#Maintenant retournons à la moyenne pondérée des attaquants
#i=0
if i==0:
    ATK_stats = ATK_stats.multiply(known_ATK.time_ratio,axis = 0)
    i=1

In [164]:
# On ajoute ATK_stats à known_ATK
if known_ATK.shape[1]==5:
    known_ATK = pd.concat([known_ATK , ATK_stats],axis = 1)
known_ATK.shape

(224873, 27)

In [165]:
# On fait la somme des stats pondérées des attaquants par équipes et par match
known_ATK = known_ATK.groupby(['game_id','player_club_id'])[np.append(['time_ratio'],ATK_stats.columns)].sum().reset_index()

In [166]:
# Puis on divise par la somme des time_ratios (toujours la moyenne pondérée):AT
known_ATK.loc[:,ATK_stats.columns] = known_ATK.loc[:,ATK_stats.columns].divide(known_ATK.time_ratio, axis = 0)

In [167]:
# On créée le fichier d'entrainement avec les ATK:
train_ATK = train1.loc[:,['game_id','player_club_id','attack']].copy()

In [168]:
train_ATK = pd.merge(train_ATK , known_ATK.drop(columns = ['time_ratio']), \
         on = ['player_club_id','game_id'], how = 'left')

In [171]:
# Reste deux cas de figures: soit un match n'a pas d'attaquant (c'est possible!), soit on ne connait pas les attaquants!
#S'il n'y a pas d'attaquants, alors attack=0 et on met les stats à 0
train_ATK.loc[train_ATK.attack == 0,ATK_stats.columns]=0
train_ATK.loc[train_ATK.attack == 0,ATK_stats.columns].head(1)

,ATK_overall,ATK_potential,ATK_pace,ATK_shooting,ATK_passing,ATK_dribbling,ATK_defending,ATK_physic,ATK_attacking_crossing,ATK_attacking_heading_accuracy,ATK_skill_curve,ATK_skill_fk_accuracy,ATK_movement_agility,ATK_movement_reactions,ATK_movement_balance,ATK_power_shot_power,ATK_power_jumping,ATK_power_stamina,ATK_mentality_aggression,ATK_mentality_positioning,ATK_mentality_penalties,ATK_mentality_composure
5160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [172]:
#Pour les matchs où on ne connait aucun attaquant:
index = train_ATK.loc[train_ATK.ATK_overall.isnull(),:].index
train_ATK.loc[index,ATK_stats.columns] = avg_ATK.values
train_ATK.loc[index,:].head(2)

,game_id,player_club_id,attack,ATK_overall,ATK_potential,ATK_pace,ATK_shooting,ATK_passing,ATK_dribbling,ATK_defending,ATK_physic,ATK_attacking_crossing,ATK_attacking_heading_accuracy,ATK_skill_curve,ATK_skill_fk_accuracy,ATK_movement_agility,ATK_movement_reactions,ATK_movement_balance,ATK_power_shot_power,ATK_power_jumping,ATK_power_stamina,ATK_mentality_aggression,ATK_mentality_positioning,ATK_mentality_penalties,ATK_mentality_composure
654,2460634,39,1.000000,69.327025,73.932658,74.459947,66.93662,60.317342,69.628081,34.18794,63.613996,57.855194,59.345951,60.089349,51.421655,72.400968,65.490317,69.203345,70.073063,65.349472,65.803697,54.299296,68.134243,63.202905,64.952025
1487,2469526,1465,2.000122,69.327025,73.932658,74.459947,66.93662,60.317342,69.628081,34.18794,63.613996,57.855194,59.345951,60.089349,51.421655,72.400968,65.490317,69.203345,70.073063,65.349472,65.803697,54.299296,68.134243,63.202905,64.952025


In [173]:
train_ATK.to_csv('../data/training_data/train_ATK6.csv')
train_ATK.to_csv('../experimentation/data/training_data/train_ATK6.csv')

# Stats des défenseurs

In [174]:
known_DEF = appearance_known_players.loc[appearance_known_players.position == 'defender',
                                        ['game_id','player_club_id', 'time_ratio','which_fifa','player_id']].reset_index(drop = True).copy()


In [175]:
DEF_stats = known_DEF.progress_apply(lambda x: player_stats(x)
                          ,axis = 1
                         )
DEF_stats = pd.concat(DEF_stats.values.tolist()).reset_index(drop = True)

100%|██████████| 266843/266843 [08:09<00:00, 544.96it/s]


In [176]:
DEF_stats.rename(columns = {x:'DEF_'+x for x in DEF_stats.columns}, inplace = True)
DEF_stats.head(1)

,DEF_overall,DEF_potential,DEF_pace,DEF_shooting,DEF_passing,DEF_dribbling,DEF_defending,DEF_physic,DEF_attacking_crossing,DEF_attacking_heading_accuracy,DEF_skill_curve,DEF_skill_fk_accuracy,DEF_movement_agility,DEF_movement_reactions,DEF_movement_balance,DEF_power_shot_power,DEF_power_jumping,DEF_power_stamina,DEF_mentality_aggression,DEF_mentality_positioning,DEF_mentality_penalties,DEF_mentality_composure
0,68,74,72.0,62.0,66.0,67.0,66.0,68.0,68,61,65,48,68,69,68,65,67,75,74,65,50,NaN


In [178]:
best_DEF = DEF_stats.rename(columns = {x:'best_'+x for x in DEF_stats.columns}).copy()

# On ajoute DEF_stats_raw à known_DEF
if best_DEF.shape[1] == 22:
    best_DEF = pd.concat([known_DEF.drop(columns=['which_fifa','player_id']) , best_DEF],axis = 1)
best_DEF.head(1)

,game_id,player_club_id,time_ratio,best_DEF_overall,best_DEF_potential,best_DEF_pace,best_DEF_shooting,best_DEF_passing,best_DEF_dribbling,best_DEF_defending,best_DEF_physic,best_DEF_attacking_crossing,best_DEF_attacking_heading_accuracy,best_DEF_skill_curve,best_DEF_skill_fk_accuracy,best_DEF_movement_agility,best_DEF_movement_reactions,best_DEF_movement_balance,best_DEF_power_shot_power,best_DEF_power_jumping,best_DEF_power_stamina,best_DEF_mentality_aggression,best_DEF_mentality_positioning,best_DEF_mentality_penalties,best_DEF_mentality_composure
0,2478588,800,1.0,68,74,72.0,62.0,66.0,67.0,66.0,68.0,68,61,65,48,68,69,68,65,67,75,74,65,50,NaN


In [181]:
#selection des meilleur scores d'overall par match et par équipe
id_best = best_DEF.groupby(['game_id','player_club_id'])['best_DEF_overall'].transform(max) == best_DEF.best_DEF_overall
best_DEF = best_DEF[id_best]
#maintenant on fait un groupby.first() pour prendre la première ligne de chaque groupe
best_DEF = best_DEF.groupby(['game_id','player_club_id']).first().reset_index()
# et on le merge avec train, on drop time_ratio 
train_best_DEF = train1.loc[:,['game_id','player_club_id','defense']].copy()
train_best_DEF = pd.merge(train_best_DEF , best_DEF.drop(columns = ['time_ratio']), \
         on = ['player_club_id','game_id'], how = 'left')

In [184]:
#defense=0 => on met les stats à 0
train_best_DEF.loc[train_best_DEF.defense == 0,train_best_DEF.columns[3:]]=0
display(train_best_DEF.loc[train_best_DEF.defense == 0,:].head(1))

#Pour les matchs où on ne connait aucun attaquant:
index = train_best_DEF.loc[train_best_DEF.best_DEF_overall.isnull(),:].index
train_best_DEF.loc[index,train_best_DEF.columns[3:]] = avg_DEF.values
train_best_DEF.loc[index,:].head(2)

,game_id,player_club_id,defense,best_DEF_overall,best_DEF_potential,best_DEF_pace,best_DEF_shooting,best_DEF_passing,best_DEF_dribbling,best_DEF_defending,best_DEF_physic,best_DEF_attacking_crossing,best_DEF_attacking_heading_accuracy,best_DEF_skill_curve,best_DEF_skill_fk_accuracy,best_DEF_movement_agility,best_DEF_movement_reactions,best_DEF_movement_balance,best_DEF_power_shot_power,best_DEF_power_jumping,best_DEF_power_stamina,best_DEF_mentality_aggression,best_DEF_mentality_positioning,best_DEF_mentality_penalties,best_DEF_mentality_composure
52863,3432158,4294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,game_id,player_club_id,defense,best_DEF_overall,best_DEF_potential,best_DEF_pace,best_DEF_shooting,best_DEF_passing,best_DEF_dribbling,best_DEF_defending,best_DEF_physic,best_DEF_attacking_crossing,best_DEF_attacking_heading_accuracy,best_DEF_skill_curve,best_DEF_skill_fk_accuracy,best_DEF_movement_agility,best_DEF_movement_reactions,best_DEF_movement_balance,best_DEF_power_shot_power,best_DEF_power_jumping,best_DEF_power_stamina,best_DEF_mentality_aggression,best_DEF_mentality_positioning,best_DEF_mentality_penalties,best_DEF_mentality_composure
5992,2494372,653,5.0,69.174201,73.719956,65.916942,43.044469,56.354282,60.15068,67.382212,70.403528,54.443954,63.838662,47.562293,40.278207,61.214259,64.886806,61.652334,55.409408,70.729511,68.619258,68.456817,47.092245,44.681,62.214994
7050,2517535,339,3.0,69.174201,73.719956,65.916942,43.044469,56.354282,60.15068,67.382212,70.403528,54.443954,63.838662,47.562293,40.278207,61.214259,64.886806,61.652334,55.409408,70.729511,68.619258,68.456817,47.092245,44.681,62.214994


In [187]:
train_best_DEF.drop(columns=['best_DEF_mentality_composure'], inplace = True)

In [188]:
train_best_DEF.to_csv('../experimentation/data/training_data/train_best_DEF6.csv')

In [189]:
DEF_stats = DEF_stats.multiply(known_DEF.time_ratio,axis = 0)

In [190]:
if known_DEF.shape[1]==5:
    known_DEF = pd.concat([known_DEF , DEF_stats],axis = 1)

In [191]:
known_DEF = known_DEF.groupby(['game_id','player_club_id'])[np.append(['time_ratio'],DEF_stats.columns)].sum().reset_index()

In [192]:
known_DEF.loc[:,DEF_stats.columns] = known_DEF.loc[:,DEF_stats.columns].divide(known_DEF.time_ratio, axis = 0)

In [193]:
train_DEF = train1.loc[:,['game_id','player_club_id','defense']].copy()

In [194]:
train_DEF = pd.merge(train_DEF , known_DEF.drop(columns = ['time_ratio']), \
         on = ['player_club_id','game_id'], how = 'left')


In [195]:
train_DEF.loc[train_DEF.defense == 0,DEF_stats.columns]=0
train_DEF.loc[train_DEF.defense == 0,:].head(1)

,game_id,player_club_id,defense,DEF_overall,DEF_potential,DEF_pace,DEF_shooting,DEF_passing,DEF_dribbling,DEF_defending,DEF_physic,DEF_attacking_crossing,DEF_attacking_heading_accuracy,DEF_skill_curve,DEF_skill_fk_accuracy,DEF_movement_agility,DEF_movement_reactions,DEF_movement_balance,DEF_power_shot_power,DEF_power_jumping,DEF_power_stamina,DEF_mentality_aggression,DEF_mentality_positioning,DEF_mentality_penalties,DEF_mentality_composure
52863,3432158,4294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [196]:
index = train_DEF.loc[train_DEF.DEF_overall.isnull(),:].index
train_DEF.loc[index,DEF_stats.columns] = avg_DEF.values
train_DEF.loc[index,:].head()

,game_id,player_club_id,defense,DEF_overall,DEF_potential,DEF_pace,DEF_shooting,DEF_passing,DEF_dribbling,DEF_defending,DEF_physic,DEF_attacking_crossing,DEF_attacking_heading_accuracy,DEF_skill_curve,DEF_skill_fk_accuracy,DEF_movement_agility,DEF_movement_reactions,DEF_movement_balance,DEF_power_shot_power,DEF_power_jumping,DEF_power_stamina,DEF_mentality_aggression,DEF_mentality_positioning,DEF_mentality_penalties,DEF_mentality_composure
5992,2494372,653,5.000000,69.174201,73.719956,65.916942,43.044469,56.354282,60.15068,67.382212,70.403528,54.443954,63.838662,47.562293,40.278207,61.214259,64.886806,61.652334,55.409408,70.729511,68.619258,68.456817,47.092245,44.681,62.214994
7050,2517535,339,3.000000,69.174201,73.719956,65.916942,43.044469,56.354282,60.15068,67.382212,70.403528,54.443954,63.838662,47.562293,40.278207,61.214259,64.886806,61.652334,55.409408,70.729511,68.619258,68.456817,47.092245,44.681,62.214994
14210,2611506,6418,5.000000,69.174201,73.719956,65.916942,43.044469,56.354282,60.15068,67.382212,70.403528,54.443954,63.838662,47.562293,40.278207,61.214259,64.886806,61.652334,55.409408,70.729511,68.619258,68.456817,47.092245,44.681,62.214994
15083,2631258,6418,5.133057,69.174201,73.719956,65.916942,43.044469,56.354282,60.15068,67.382212,70.403528,54.443954,63.838662,47.562293,40.278207,61.214259,64.886806,61.652334,55.409408,70.729511,68.619258,68.456817,47.092245,44.681,62.214994
15648,2646302,2451,1.000000,69.174201,73.719956,65.916942,43.044469,56.354282,60.15068,67.382212,70.403528,54.443954,63.838662,47.562293,40.278207,61.214259,64.886806,61.652334,55.409408,70.729511,68.619258,68.456817,47.092245,44.681,62.214994


In [197]:
train_DEF.to_csv('../experimentation/data/training_data/train_DEF6.csv')

# Stats des milieux de terrain

In [198]:
known_MF = appearance_known_players.loc[appearance_known_players.position == 'midfield',
                                        ['game_id','player_club_id', 'time_ratio','which_fifa','player_id']].reset_index(drop = True).copy()


In [199]:
MF_stats = known_MF.progress_apply(lambda x: player_stats(x)
                          ,axis = 1
                         )
MF_stats = pd.concat(MF_stats.values.tolist()).reset_index(drop = True)

100%|██████████| 246963/246963 [07:32<00:00, 545.59it/s]


In [200]:
MF_stats.rename(columns = {x:'MF_'+x for x in MF_stats.columns}, inplace = True)
MF_stats.head(1)

,MF_overall,MF_potential,MF_pace,MF_shooting,MF_passing,MF_dribbling,MF_defending,MF_physic,MF_attacking_crossing,MF_attacking_heading_accuracy,MF_skill_curve,MF_skill_fk_accuracy,MF_movement_agility,MF_movement_reactions,MF_movement_balance,MF_power_shot_power,MF_power_jumping,MF_power_stamina,MF_mentality_aggression,MF_mentality_positioning,MF_mentality_penalties,MF_mentality_composure
0,82,82,66.0,81.0,84.0,79.0,52.0,70.0,86,60,87,89,72,78,61,82,70,86,57,80,80,81.0


In [201]:
best_MF = MF_stats.rename(columns = {x:'best_'+x for x in MF_stats.columns}).copy()

# On ajoute MF_stats_raw à known_MF
if best_MF.shape[1] == 22:
    best_MF = pd.concat([known_MF.drop(columns=['which_fifa','player_id']) , best_MF],axis = 1)
best_MF.head(1)

,game_id,player_club_id,time_ratio,best_MF_overall,best_MF_potential,best_MF_pace,best_MF_shooting,best_MF_passing,best_MF_dribbling,best_MF_defending,best_MF_physic,best_MF_attacking_crossing,best_MF_attacking_heading_accuracy,best_MF_skill_curve,best_MF_skill_fk_accuracy,best_MF_movement_agility,best_MF_movement_reactions,best_MF_movement_balance,best_MF_power_shot_power,best_MF_power_jumping,best_MF_power_stamina,best_MF_mentality_aggression,best_MF_mentality_positioning,best_MF_mentality_penalties,best_MF_mentality_composure
0,2460300,2288,1.0,82,82,66.0,81.0,84.0,79.0,52.0,70.0,86,60,87,89,72,78,61,82,70,86,57,80,80,81.0


In [202]:
#selection des meilleur scores d'overall par match et par équipe
id_best = best_MF.groupby(['game_id','player_club_id'])['best_MF_overall'].transform(max) == best_MF.best_MF_overall
best_MF = best_MF[id_best]
#maintenant on fait un groupby.first() pour prendre la première ligne de chaque groupe
best_MF = best_MF.groupby(['game_id','player_club_id']).first().reset_index()
# et on le merge avec train, on drop time_ratio 
train_best_MF = train1.loc[:,['game_id','player_club_id','midfield']].copy()
train_best_MF = pd.merge(train_best_MF , best_MF.drop(columns = ['time_ratio']), \
         on = ['player_club_id','game_id'], how = 'left')

In [204]:
#midfield=0 => on met les stats à 0
train_best_MF.loc[train_best_MF.midfield == 0,train_best_MF.columns[3:]]=0
display(train_best_MF.loc[train_best_MF.midfield == 0,:].head(1))

#Pour les matchs où on ne connait aucun MF:
index = train_best_MF.loc[train_best_MF.best_MF_overall.isnull(),:].index
train_best_MF.loc[index,train_best_MF.columns[3:]] = avg_MID.values
train_best_MF.loc[index,:].head(2)

,game_id,player_club_id,midfield,best_MF_overall,best_MF_potential,best_MF_pace,best_MF_shooting,best_MF_passing,best_MF_dribbling,best_MF_defending,best_MF_physic,best_MF_attacking_crossing,best_MF_attacking_heading_accuracy,best_MF_skill_curve,best_MF_skill_fk_accuracy,best_MF_movement_agility,best_MF_movement_reactions,best_MF_movement_balance,best_MF_power_shot_power,best_MF_power_jumping,best_MF_power_stamina,best_MF_mentality_aggression,best_MF_mentality_positioning,best_MF_mentality_penalties,best_MF_mentality_composure
9914,2581814,18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,game_id,player_club_id,midfield,best_MF_overall,best_MF_potential,best_MF_pace,best_MF_shooting,best_MF_passing,best_MF_dribbling,best_MF_defending,best_MF_physic,best_MF_attacking_crossing,best_MF_attacking_heading_accuracy,best_MF_skill_curve,best_MF_skill_fk_accuracy,best_MF_movement_agility,best_MF_movement_reactions,best_MF_movement_balance,best_MF_power_shot_power,best_MF_power_jumping,best_MF_power_stamina,best_MF_mentality_aggression,best_MF_mentality_positioning,best_MF_mentality_penalties,best_MF_mentality_composure
986,2460800,24,1.0,69.246334,74.268119,65.762044,59.722245,66.481357,69.112275,58.764139,66.000838,60.538752,54.977377,60.99623,56.488898,70.036866,66.411395,70.603687,66.884374,63.58651,70.992459,64.35442,61.945538,56.255551,66.7323
2631,2475409,3368,1.0,69.246334,74.268119,65.762044,59.722245,66.481357,69.112275,58.764139,66.000838,60.538752,54.977377,60.99623,56.488898,70.036866,66.411395,70.603687,66.884374,63.58651,70.992459,64.35442,61.945538,56.255551,66.7323


In [207]:
train_best_MF.drop(columns = ['best_MF_mentality_composure'], inplace = True)

In [208]:
train_best_MF.to_csv('../experimentation/data/training_data/train_best_MF6.csv')

In [209]:
MF_stats = MF_stats.multiply(known_MF.time_ratio,axis = 0)

In [210]:
if known_MF.shape[1]==5:
    known_MF = pd.concat([known_MF , MF_stats],axis = 1)

In [211]:
known_MF = known_MF.groupby(['game_id','player_club_id'])[np.append(['time_ratio'],MF_stats.columns)].sum().reset_index()

In [212]:
known_MF.loc[:,MF_stats.columns] = known_MF.loc[:,MF_stats.columns].divide(known_MF.time_ratio, axis = 0)

In [213]:
train_MF = train1.loc[:,['game_id','player_club_id','midfield']].copy()

In [214]:
train_MF = pd.merge(train_MF , known_MF.drop(columns = ['time_ratio']), \
         on = ['player_club_id','game_id'], how = 'left')


In [215]:
train_MF.loc[train_MF.midfield == 0,MF_stats.columns]=0
train_MF.loc[train_MF.midfield == 0,:].head(1)

,game_id,player_club_id,midfield,MF_overall,MF_potential,MF_pace,MF_shooting,MF_passing,MF_dribbling,MF_defending,MF_physic,MF_attacking_crossing,MF_attacking_heading_accuracy,MF_skill_curve,MF_skill_fk_accuracy,MF_movement_agility,MF_movement_reactions,MF_movement_balance,MF_power_shot_power,MF_power_jumping,MF_power_stamina,MF_mentality_aggression,MF_mentality_positioning,MF_mentality_penalties,MF_mentality_composure
9914,2581814,18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [216]:
index = train_MF.loc[train_MF.MF_overall.isnull(),:].index
train_MF.loc[index,MF_stats.columns] = avg_MID.values
train_MF.loc[index,:].head() 

,game_id,player_club_id,midfield,MF_overall,MF_potential,MF_pace,MF_shooting,MF_passing,MF_dribbling,MF_defending,MF_physic,MF_attacking_crossing,MF_attacking_heading_accuracy,MF_skill_curve,MF_skill_fk_accuracy,MF_movement_agility,MF_movement_reactions,MF_movement_balance,MF_power_shot_power,MF_power_jumping,MF_power_stamina,MF_mentality_aggression,MF_mentality_positioning,MF_mentality_penalties,MF_mentality_composure
986,2460800,24,1.000000,69.246334,74.268119,65.762044,59.722245,66.481357,69.112275,58.764139,66.000838,60.538752,54.977377,60.99623,56.488898,70.036866,66.411395,70.603687,66.884374,63.58651,70.992459,64.35442,61.945538,56.255551,66.7323
2631,2475409,3368,1.000000,69.246334,74.268119,65.762044,59.722245,66.481357,69.112275,58.764139,66.000838,60.538752,54.977377,60.99623,56.488898,70.036866,66.411395,70.603687,66.884374,63.58651,70.992459,64.35442,61.945538,56.255551,66.7323
2725,2475456,3368,0.999756,69.246334,74.268119,65.762044,59.722245,66.481357,69.112275,58.764139,66.000838,60.538752,54.977377,60.99623,56.488898,70.036866,66.411395,70.603687,66.884374,63.58651,70.992459,64.35442,61.945538,56.255551,66.7323
3875,2480908,6414,2.144531,69.246334,74.268119,65.762044,59.722245,66.481357,69.112275,58.764139,66.000838,60.538752,54.977377,60.99623,56.488898,70.036866,66.411395,70.603687,66.884374,63.58651,70.992459,64.35442,61.945538,56.255551,66.7323
5797,2491049,6414,2.088867,69.246334,74.268119,65.762044,59.722245,66.481357,69.112275,58.764139,66.000838,60.538752,54.977377,60.99623,56.488898,70.036866,66.411395,70.603687,66.884374,63.58651,70.992459,64.35442,61.945538,56.255551,66.7323


In [219]:
train_MF.to_csv('../experimentation/data/training_data/train_MF6.csv')

# Création du train dataset

In [220]:
train_GK.head(1)

,game_id,player_club_id,GK,GK_overall,GK_potential,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed
0,2457642,610,1.0,77.0,77.0,25.0,25.0,25.0,25.0,68.0,79.0,58.0,23.0,82.0,42.0,34.0,25.0,29.0,82.0,58.0


In [221]:
train = pd.merge(train_GK , train_ATK, \
         on = ['player_club_id','game_id'], how = 'inner')


In [222]:
train = pd.merge(train , train_DEF, \
         on = ['player_club_id','game_id'], how = 'inner')
train = pd.merge(train , train_MF, \
         on = ['player_club_id','game_id'], how = 'inner')
train.head(1)

,game_id,player_club_id,GK,GK_overall,GK_potential,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed,attack,ATK_overall,ATK_potential,ATK_pace,ATK_shooting,ATK_passing,ATK_dribbling,ATK_defending,ATK_physic,ATK_attacking_crossing,ATK_attacking_heading_accuracy,ATK_skill_curve,ATK_skill_fk_accuracy,ATK_movement_agility,ATK_movement_reactions,ATK_movement_balance,ATK_power_shot_power,ATK_power_jumping,ATK_power_stamina,ATK_mentality_aggression,ATK_mentality_positioning,ATK_mentality_penalties,ATK_mentality_composure,defense,DEF_overall,DEF_potential,DEF_pace,DEF_shooting,DEF_passing,DEF_dribbling,DEF_defending,DEF_physic,DEF_attacking_crossing,DEF_attacking_heading_accuracy,DEF_skill_curve,DEF_skill_fk_accuracy,DEF_movement_agility,DEF_movement_reactions,DEF_movement_balance,DEF_power_shot_power,DEF_power_jumping,DEF_power_stamina,DEF_mentality_aggression,DEF_mentality_positioning,DEF_mentality_penalties,DEF_mentality_composure,midfield,MF_overall,MF_potential,MF_pace,MF_shooting,MF_passing,MF_dribbling,MF_defending,MF_physic,MF_attacking_crossing,MF_attacking_heading_accuracy,MF_skill_curve,MF_skill_fk_accuracy,MF_movement_agility,MF_movement_reactions,MF_movement_balance,MF_power_shot_power,MF_power_jumping,MF_power_stamina,MF_mentality_aggression,MF_mentality_positioning,MF_mentality_penalties,MF_mentality_composure
0,2457642,610,1.0,77.0,77.0,25.0,25.0,25.0,25.0,68.0,79.0,58.0,23.0,82.0,42.0,34.0,25.0,29.0,82.0,58.0,3.000244,71.009603,82.410645,75.825439,62.260417,65.342611,72.450928,32.920817,58.145671,63.413249,62.345378,67.527995,55.281657,75.328532,66.231445,66.635254,68.820801,59.248698,63.693848,45.52946,63.178467,55.97819,0.0,4.43335,72.123844,76.025881,72.635077,43.77957,61.192126,67.447412,71.191575,70.417181,55.523293,72.567401,48.50859,31.389262,66.485077,70.19163,64.966355,61.838711,78.30391,73.003524,70.063822,55.320264,40.005341,0.0,2.566895,75.183029,78.79414,73.495434,71.375761,71.571918,76.118721,54.801941,65.583143,62.321918,60.984209,68.908866,59.802892,75.327055,75.23516,73.483067,73.557268,63.364536,78.833904,59.296613,74.741248,67.101979,0.0


In [223]:
train.to_csv('../experimentation/data/training_data/train6.csv')

# Générer le train.csv

In [116]:
# il faut réussir à diviser le dataset en home et away, puis faire un join.

In [230]:
#repérons les club_id qui n'existent pas
club_IDs = clubs.club_id.unique()
train.loc[~train.player_club_id.isin(club_IDs),'player_club_id'].unique()

array([-25110, -25814, -24335, -24305,  -4587, -24262, -15834])

In [238]:
tt = train.loc[train.player_club_id == -15834,'game_id'].values[2]
games.loc[games.game_id==tt,:]

,game_id,competition_code,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,away_club_position,stadium,attendance,referee,url
40682,3602415,RU1,2021,3. Matchday,2021-08-07,232,49702,1,2,11.0,5.0,Otkrytie Bank Arena,2849.0,Aleksey Sukhoy,https://www.transfermarkt.co.uk/spartak-moscow...


1. -25110 => 40426
2. -25814 => 39722
3. -24335 => 41201
4. -24305 => 41231
5. -24262 => 41274
6. -4587 => 60949  
7. -15834 => 49702

In [239]:
#on a repéré quelle mauvaise notation correspondait à qui, maintenant changeons les valeurs:
train.loc[train.player_club_id == -25110,'player_club_id'] = 40426
train.loc[train.player_club_id == -25814,'player_club_id'] = 39722
train.loc[train.player_club_id == -24335,'player_club_id'] = 41201
train.loc[train.player_club_id == -24305,'player_club_id'] = 41231
train.loc[train.player_club_id == -24262,'player_club_id'] = 41274
train.loc[train.player_club_id == -4587,'player_club_id'] = 60949
train.loc[train.player_club_id == -15834,'player_club_id'] = 49702

In [240]:
#maintenant c'est bon!
train.loc[~train.player_club_id.isin(club_IDs),'player_club_id'].unique()

array([], dtype=int64)

In [241]:
def HomeOrAway(x):
    if games.loc[games.game_id == x['game_id'],'home_club_id'].values[0] == x['player_club_id']:
        return 'Home'
    elif games.loc[games.game_id == x['game_id'],'away_club_id'].values[0] == x['player_club_id']:
        return 'Away'
    else:
        return 'problem'    

In [242]:
train['HomeOrAway']=None
train['HomeOrAway'] = train.progress_apply(HomeOrAway, axis = 1)


100%|██████████| 60846/60846 [00:30<00:00, 1976.44it/s]


In [243]:
train_home = train.loc[train.HomeOrAway == 'Home',:].drop(columns = ['HomeOrAway','player_club_id']).copy()
train_away = train.loc[train.HomeOrAway == 'Away',:].drop(columns = ['HomeOrAway','player_club_id']).copy()

train_home.rename(columns={x:'H_'+x for x in train_home.columns[1:]}, inplace = True)
train_away.rename(columns={x:'A_'+x for x in train_away.columns[1:]}, inplace = True)

train_final = pd.merge(train_home , train_away, on = ['game_id'], how = 'inner')
train_final.head()

In [221]:
x = train_final.game_id[4]
games.loc[games.game_id==x,'home_club_goals'].values[0]-games.loc[games.game_id==x,'away_club_goals'].values[0]

-1

In [246]:
train_final['Home_result'] = train_final.game_id.progress_apply(lambda x:
                                                                games.loc[games.game_id==x,'home_club_goals'].values[0]-games.loc[games.game_id==x,'away_club_goals'].values[0]
                                                               )

100%|██████████| 30423/30423 [00:16<00:00, 1835.09it/s]


In [248]:
train_final.head(1)

,game_id,H_GK,H_GK_overall,H_GK_potential,H_GK_attacking_crossing,H_GK_attacking_heading_accuracy,H_GK_skill_curve,H_GK_skill_fk_accuracy,H_GK_movement_agility,H_GK_movement_reactions,H_GK_movement_balance,H_GK_power_shot_power,H_GK_power_jumping,H_GK_power_stamina,H_GK_mentality_aggression,H_GK_mentality_positioning,H_GK_mentality_penalties,H_GK_goalkeeping_diving,H_GK_goalkeeping_speed,H_attack,H_ATK_overall,H_ATK_potential,H_ATK_pace,H_ATK_shooting,H_ATK_passing,H_ATK_dribbling,H_ATK_defending,H_ATK_physic,H_ATK_attacking_crossing,H_ATK_attacking_heading_accuracy,H_ATK_skill_curve,H_ATK_skill_fk_accuracy,H_ATK_movement_agility,H_ATK_movement_reactions,H_ATK_movement_balance,H_ATK_power_shot_power,H_ATK_power_jumping,H_ATK_power_stamina,H_ATK_mentality_aggression,H_ATK_mentality_positioning,H_ATK_mentality_penalties,H_ATK_mentality_composure,H_defense,H_DEF_overall,H_DEF_potential,H_DEF_pace,H_DEF_shooting,H_DEF_passing,H_DEF_dribbling,H_DEF_defending,H_DEF_physic,H_DEF_attacking_crossing,H_DEF_attacking_heading_accuracy,H_DEF_skill_curve,H_DEF_skill_fk_accuracy,H_DEF_movement_agility,H_DEF_movement_reactions,H_DEF_movement_balance,H_DEF_power_shot_power,H_DEF_power_jumping,H_DEF_power_stamina,H_DEF_mentality_aggression,H_DEF_mentality_positioning,H_DEF_mentality_penalties,H_DEF_mentality_composure,H_midfield,H_MF_overall,H_MF_potential,H_MF_pace,H_MF_shooting,H_MF_passing,H_MF_dribbling,H_MF_defending,H_MF_physic,H_MF_attacking_crossing,H_MF_attacking_heading_accuracy,H_MF_skill_curve,H_MF_skill_fk_accuracy,H_MF_movement_agility,H_MF_movement_reactions,H_MF_movement_balance,H_MF_power_shot_power,H_MF_power_jumping,H_MF_power_stamina,H_MF_mentality_aggression,H_MF_mentality_positioning,H_MF_mentality_penalties,H_MF_mentality_composure,A_GK,A_GK_overall,A_GK_potential,A_GK_attacking_crossing,A_GK_attacking_heading_accuracy,A_GK_skill_curve,A_GK_skill_fk_accuracy,A_GK_movement_agility,A_GK_movement_reactions,A_GK_movement_balance,A_GK_power_shot_power,A_GK_power_jumping,A_GK_power_stamina,A_GK_mentality_aggression,A_GK_mentality_positioning,A_GK_mentality_penalties,A_GK_goalkeeping_diving,A_GK_goalkeeping_speed,A_attack,A_ATK_overall,A_ATK_potential,A_ATK_pace,A_ATK_shooting,A_ATK_passing,A_ATK_dribbling,A_ATK_defending,A_ATK_physic,A_ATK_attacking_crossing,A_ATK_attacking_heading_accuracy,A_ATK_skill_curve,A_ATK_skill_fk_accuracy,A_ATK_movement_agility,A_ATK_movement_reactions,A_ATK_movement_balance,A_ATK_power_shot_power,A_ATK_power_jumping,A_ATK_power_stamina,A_ATK_mentality_aggression,A_ATK_mentality_positioning,A_ATK_mentality_penalties,A_ATK_mentality_composure,A_defense,A_DEF_overall,A_DEF_potential,A_DEF_pace,A_DEF_shooting,A_DEF_passing,A_DEF_dribbling,A_DEF_defending,A_DEF_physic,A_DEF_attacking_crossing,A_DEF_attacking_heading_accuracy,A_DEF_skill_curve,A_DEF_skill_fk_accuracy,A_DEF_movement_agility,A_DEF_movement_reactions,A_DEF_movement_balance,A_DEF_power_shot_power,A_DEF_power_jumping,A_DEF_power_stamina,A_DEF_mentality_aggression,A_DEF_mentality_positioning,A_DEF_mentality_penalties,A_DEF_mentality_composure,A_midfield,A_MF_overall,A_MF_potential,A_MF_pace,A_MF_shooting,A_MF_passing,A_MF_dribbling,A_MF_defending,A_MF_physic,A_MF_attacking_crossing,A_MF_attacking_heading_accuracy,A_MF_skill_curve,A_MF_skill_fk_accuracy,A_MF_movement_agility,A_MF_movement_reactions,A_MF_movement_balance,A_MF_power_shot_power,A_MF_power_jumping,A_MF_power_stamina,A_MF_mentality_aggression,A_MF_mentality_positioning,A_MF_mentality_penalties,A_MF_mentality_composure,Home_result
0,2457642,1.0,70.0,70.0,25.0,25.0,25.0,25.0,36.0,65.0,41.0,27.0,45.0,40.0,30.0,25.0,45.0,68.0,38.0,0.55542,63.532074,73.407733,72.768014,59.853691,57.254833,65.331283,26.988137,61.213093,61.013181,57.174868,53.896309,34.744728,64.211775,47.459139,60.733304,66.93058,53.176626,68.050088,64.891476,58.294376,62.172671,0.0,4.0,65.75,68.0,62.0,51.5,59.75,63.0,66.0,68.0,50.5,61.75,61.25,52.0,58.75,64.0,64.5,62.5,64.75,68.5,59.25,53.5,64.25,0.0,5.444336,69.191714,74.

In [249]:
train_final = train_final.astype(dtype={x: np.float16 for x in train_final.columns[1:-1]})
train_final.to_csv('../experimentation/data/training_data/train_final_6.csv')

# train_best_players.csv

In [251]:
train_best_ATK.shape, train_best_DEF.shape, train_best_MF.shape,

((60846, 24), (60846, 24), (60846, 24))

In [256]:
train_best_ATK.drop(columns = ['attack'], inplace = True)
train_best_DEF.drop(columns = ['defense'], inplace = True)
train_best_MF.drop(columns = ['midfield'], inplace = True)

In [260]:
train_best = pd.merge(train_best_ATK,train_best_DEF,on=['game_id','player_club_id'])
train_best = pd.merge(train_best,train_best_MF,on=['game_id','player_club_id'])


In [264]:
print(train_best.loc[~train_best.player_club_id.isin(club_IDs),'player_club_id'].unique())
#on a repéré quelle mauvaise notation correspondait à qui, maintenant changeons les valeurs:
train_best.loc[train_best.player_club_id == -25110,'player_club_id'] = 40426
train_best.loc[train_best.player_club_id == -25814,'player_club_id'] = 39722
train_best.loc[train_best.player_club_id == -24335,'player_club_id'] = 41201
train_best.loc[train_best.player_club_id == -24305,'player_club_id'] = 41231
train_best.loc[train_best.player_club_id == -24262,'player_club_id'] = 41274
train_best.loc[train_best.player_club_id == -4587,'player_club_id'] = 60949
train_best.loc[train_best.player_club_id == -15834,'player_club_id'] = 49702

train_best.loc[~train_best.player_club_id.isin(club_IDs),'player_club_id'].unique()


[-15834]


array([], dtype=int64)

In [265]:
train_best['HomeOrAway']=None
train_best['HomeOrAway'] = train_best.progress_apply(HomeOrAway, axis = 1)


100%|██████████| 60846/60846 [00:29<00:00, 2083.96it/s]


In [266]:
train_best_home = train_best.loc[train_best.HomeOrAway == 'Home',:].drop(columns = ['HomeOrAway','player_club_id']).copy()
train_best_away = train_best.loc[train_best.HomeOrAway == 'Away',:].drop(columns = ['HomeOrAway','player_club_id']).copy()

train_best_home.rename(columns={x:'H_'+x for x in train_best_home.columns[1:]}, inplace = True)
train_best_away.rename(columns={x:'A_'+x for x in train_best_away.columns[1:]}, inplace = True)

train_best_final = pd.merge(train_best_home , train_best_away, on = ['game_id'], how = 'inner')
train_best_final.head()

,game_id,H_best_ATK_overall,H_best_ATK_potential,H_best_ATK_pace,H_best_ATK_shooting,H_best_ATK_passing,H_best_ATK_dribbling,H_best_ATK_defending,H_best_ATK_physic,H_best_ATK_attacking_crossing,H_best_ATK_attacking_heading_accuracy,H_best_ATK_skill_curve,H_best_ATK_skill_fk_accuracy,H_best_ATK_movement_agility,H_best_ATK_movement_reactions,H_best_ATK_movement_balance,H_best_ATK_power_shot_power,H_best_ATK_power_jumping,H_best_ATK_power_stamina,H_best_ATK_mentality_aggression,H_best_ATK_mentality_positioning,H_best_ATK_mentality_penalties,H_best_DEF_overall,H_best_DEF_potential,H_best_DEF_pace,H_best_DEF_shooting,H_best_DEF_passing,H_best_DEF_dribbling,H_best_DEF_defending,H_best_DEF_physic,H_best_DEF_attacking_crossing,H_best_DEF_attacking_heading_accuracy,H_best_DEF_skill_curve,H_best_DEF_skill_fk_accuracy,H_best_DEF_movement_agility,H_best_DEF_movement_reactions,H_best_DEF_movement_balance,H_best_DEF_power_shot_power,H_best_DEF_power_jumping,H_best_DEF_power_stamina,H_best_DEF_mentality_aggression,H_best_DEF_mentality_positioning,H_best_DEF_mentality_penalties,H_best_MF_overall,H_best_MF_potential,H_best_MF_pace,H_best_MF_shooting,H_best_MF_passing,H_best_MF_dribbling,H_best_MF_defending,H_best_MF_physic,H_best_MF_attacking_crossing,H_best_MF_attacking_heading_accuracy,H_best_MF_skill_curve,H_best_MF_skill_fk_accuracy,H_best_MF_movement_agility,H_best_MF_movement_reactions,H_best_MF_movement_balance,H_best_MF_power_shot_power,H_best_MF_power_jumping,H_best_MF_power_stamina,H_best_MF_mentality_aggression,H_best_MF_mentality_positioning,H_best_MF_mentality_penalties,A_best_ATK_overall,A_best_ATK_potential,A_best_ATK_pace,A_best_ATK_shooting,A_best_ATK_passing,A_best_ATK_dribbling,A_best_ATK_defending,A_best_ATK_physic,A_best_ATK_attacking_crossing,A_best_ATK_attacking_heading_accuracy,A_best_ATK_skill_curve,A_best_ATK_skill_fk_accuracy,A_best_ATK_movement_agility,A_best_ATK_movement_reactions,A_best_ATK_movement_balance,A_best_ATK_power_shot_power,A_best_ATK_power_jumping,A_best_ATK_power_stamina,A_best_ATK_mentality_aggression,A_best_ATK_mentality_positioning,A_best_ATK_mentality_penalties,A_best_DEF_overall,A_best_DEF_potential,A_best_DEF_pace,A_best_DEF_shooting,A_best_DEF_passing,A_best_DEF_dribbling,A_best_DEF_defending,A_best_DEF_physic,A_best_DEF_attacking_crossing,A_best_DEF_attacking_heading_accuracy,A_best_DEF_skill_curve,A_best_DEF_skill_fk_accuracy,A_best_DEF_movement_agility,A_best_DEF_movement_reactions,A_best_DEF_movement_balance,A_best_DEF_power_shot_power,A_best_DEF_power_jumping,A_best_DEF_power_stamina,A_best_DEF_mentality_aggression,A_best_DEF_mentality_positioning,A_best_DEF_mentality_penalties,A_best_MF_overall,A_best_MF_potential,A_best_MF_pace,A_best_MF_shooting,A_best_MF_passing,A_best_MF_dribbling,A_best_MF_defending,A_best_MF_physic,A_best_MF_attacking_crossing,A_best_MF_attacking_heading_accuracy,A_best_MF_skill_curve,A_best_MF_skill_fk_accuracy,A_best_MF_movement_agility,A_best_MF_movement_reactions,A_best_MF_movement_balance,A_best_MF_power_shot_power,A_best_MF_power_jumping,A_best_MF_power_stamina,A_best_MF_mentality_aggression,A_best_MF_mentality_positioning,A_best_MF_mentality_penalties
0,2457642,64.0,73.0,75.0,59.0,63.0,68.0,27.0,52.0,65.0,44.0,68.0,44.0,66.0,40.0,70.0,63.0,40.0,65.0,68.0,57.0,60.0,67.0,67.0,74.0,47.0,57.0,66.0,67.0,67.0,56.0,57.0,41.0,36.0,64.0,66.0,69.0,44.0,71.0,75.0,58.0,53.0,61.0,73.0,79.0,70.0,56.0,68.0,74.0,66.0,69.0,57.0,41.0,42.0,40.0,72.0,72.0,81.0,56.0,74.0,81.0,60.0,62.0,49.0,73.0,76.0,69.0,72.0,58.0,67.0,42.0,77.0,47.0,84.0,52.0,53.0,64.0,71.0,59.0,78.0,77.0,77.0,72.0,75.0,70.0,76.0,76.0,67.0,49.0,67.0,70.0,76.0,74.0,52.0,77.0,55.0,44.0,68.0,77.0,66.0,67.0,77.0,72.0,73.0,59.0,40.0,76.0,76.0,67.0,77.0,77.0,78.0,52.0,60.0,79.0,57.0,83.0,82.0,75.0,74.0,77.0,83.0,45.0,74.0,46.0,72.0,76.0
1,2458528,78.0,82.0,88.0,75.0,72.0,82.0,41.0,61.0,67.0,57.0,51.0,64.0,89.0,78.0,80.0,76.0,62.0,70.0,55.0,73.0,74.0,79.0,79.0,72.0,72.0,84.0,74.0,76.0,76.0,86.0,59.0,85.0,82.0,71.0,82.

In [268]:
train_best_final = train_best_final.astype(dtype={x: np.int8 for x in train_best_final.columns[1:]})
train_best_final.to_csv('../experimentation/data/training_data/train_best_final_6.csv')